In [2]:
import cv2
from ultralytics import YOLO
import pandas as pd
from collections import defaultdict
import time

VIDEO_PATH = "plates_test.MP4"
OUTPUT_VIDEO = "resultado.mp4"
OUTPUT_CSV = "detecciones.csv"

print("Cargando modelos...")
model_objects = YOLO('yolo11n.pt')
model_plates = YOLO('yolo11n_best.pt')

VEHICLE_CLASSES = [2, 3, 5, 7]
PERSON_CLASS = [0]
ALL_CLASSES = PERSON_CLASS + VEHICLE_CLASSES

class_counts = defaultdict(set)
detections_data = []
plate_cache = {}

def detect_plate_in_vehicle(frame, vehicle_box):
    x1, y1, x2, y2 = map(int, vehicle_box)
    h, w = frame.shape[:2]
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w, x2), min(h, y2)

    if x2 <= x1 or y2 <= y1:
        return None

    vehicle_crop = frame[y1:y2, x1:x2]
    if vehicle_crop.size == 0:
        return None

    plate_results = model_plates(vehicle_crop, verbose=False, conf=0.4, imgsz=320)

    if len(plate_results[0].boxes) > 0:
        best_idx = plate_results[0].boxes.conf.argmax()
        plate_box = plate_results[0].boxes[best_idx]
        px1, py1, px2, py2 = map(int, plate_box.xyxy[0])

        return {
            'coords': (x1+px1, y1+py1, x1+px2, y1+py2),
            'conf': float(plate_box.conf[0])
        }
    return None

cap = cv2.VideoCapture(VIDEO_PATH)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"\nVideo: {total_frames} frames, {width}x{height}, {fps}fps\n")

out = cv2.VideoWriter(OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

frame_idx = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model_objects.track(frame, persist=True, verbose=False,
                                 conf=0.4, classes=ALL_CLASSES, imgsz=640)

    if results[0].boxes is not None and len(results[0].boxes) > 0:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        classes = results[0].boxes.cls.cpu().numpy().astype(int)
        confs = results[0].boxes.conf.cpu().numpy()
        track_ids = results[0].boxes.id.cpu().numpy().astype(int) if results[0].boxes.id is not None else range(len(boxes))

        for box, cls, conf, tid in zip(boxes, classes, confs, track_ids):
            x1, y1, x2, y2 = map(int, box)
            obj_type = model_objects.names[cls]

            class_counts[obj_type].add(tid)

            plate_info = None
            if cls in VEHICLE_CLASSES:
                if frame_idx % 15 == 0:
                    plate_info = detect_plate_in_vehicle(frame, box)
                    if plate_info:
                        plate_cache[tid] = plate_info
                elif tid in plate_cache:
                    plate_info = plate_cache[tid]

            detections_data.append({
                'fotograma': frame_idx,
                'tipo_objeto': obj_type,
                'confianza': round(float(conf), 3),
                'identificador_tracking': int(tid),
                'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2,
                'matricula': 'detectada' if plate_info else '',
                'confianza_matricula': round(plate_info['conf'], 3) if plate_info else 0.0,
                'mx1': plate_info['coords'][0] if plate_info else 0,
                'my1': plate_info['coords'][1] if plate_info else 0,
                'mx2': plate_info['coords'][2] if plate_info else 0,
                'my2': plate_info['coords'][3] if plate_info else 0,
                'texto_matricula': ''
            })

            color = (0, 255, 0) if cls == 0 else (255, 0, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # ======== CAMBIO PARA TEXTO IDENTIFICADOR =========
            info_text = f"ID:{tid} {obj_type} {conf:.2f}"
            text_size, _ = cv2.getTextSize(info_text, cv2.FONT_HERSHEY_SIMPLEX, 0.55, 2)
            text_x = x1
            text_y = max(y1 - 12, text_size[1])  # Evita solapar fuera de imagen superior

            # Fondo negro debajo del texto (opcional)
            cv2.rectangle(frame, (text_x, text_y-text_size[1]-2), (text_x+text_size[0], text_y+2), (0,0,0), -1)
            cv2.putText(frame, info_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.55, color, 2)

            # Opcionalmente un texto para matrícula detectada
            if plate_info:
                px1, py1, px2, py2 = plate_info['coords']
                cv2.rectangle(frame, (px1, py1), (px2, py2), (0, 255, 255), 2)
                # Descomenta si quieres pintar confianza matrícula encima:
                # plate_text = f"Plate {plate_info['conf']:.2f}"
                # ptext_size, _ = cv2.getTextSize(plate_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
                # ptext_x, ptext_y = px1, max(py1-8, ptext_size[1])
                # cv2.rectangle(frame, (ptext_x, ptext_y-ptext_size[1]-2), (ptext_x+ptext_size[0], ptext_y+2), (0,0,0), -1)
                # cv2.putText(frame, plate_text, (ptext_x, ptext_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 2)

    y = 30
    for obj, ids in class_counts.items():
        cv2.putText(frame, f"{obj}: {len(ids)}", (10, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        y += 30

    out.write(frame)
    frame_idx += 1

    if frame_idx % 30 == 0:
        elapsed = time.time() - start_time
        fps_proc = frame_idx / elapsed
        remaining = (total_frames - frame_idx) / fps_proc
        print(f"Frame {frame_idx}/{total_frames} ({frame_idx/total_frames*100:.1f}%) | "
              f"{fps_proc:.1f} fps | ETA: {remaining/60:.1f}min")

cap.release()
out.release()
pd.DataFrame(detections_data).to_csv(OUTPUT_CSV, index=False)

total_time = time.time() - start_time
print(f"\n✅ Completado en {total_time/60:.1f} minutos")
print(f"📊 Detecciones totales en CSV: {len(detections_data)}")
print(f"🚗 Objetos únicos detectados:")
for obj, ids in sorted(class_counts.items()):
    print(f"   • {obj}: {len(ids)} objetos únicos")
print(f"🚦 Matrículas: {len(plate_cache)}")


Cargando modelos...

Video: 2832 frames, 1920x1080, 25fps

Frame 30/2832 (1.1%) | 2.4 fps | ETA: 19.4min
Frame 60/2832 (2.1%) | 3.1 fps | ETA: 14.9min
Frame 90/2832 (3.2%) | 3.6 fps | ETA: 12.7min
Frame 120/2832 (4.2%) | 3.7 fps | ETA: 12.2min
Frame 150/2832 (5.3%) | 3.7 fps | ETA: 12.2min
Frame 180/2832 (6.4%) | 3.7 fps | ETA: 11.8min
Frame 210/2832 (7.4%) | 3.8 fps | ETA: 11.4min
Frame 240/2832 (8.5%) | 3.8 fps | ETA: 11.5min
Frame 270/2832 (9.5%) | 3.8 fps | ETA: 11.3min
Frame 300/2832 (10.6%) | 3.8 fps | ETA: 11.1min
Frame 330/2832 (11.7%) | 3.8 fps | ETA: 10.8min
Frame 360/2832 (12.7%) | 3.9 fps | ETA: 10.7min
Frame 390/2832 (13.8%) | 3.9 fps | ETA: 10.4min
Frame 420/2832 (14.8%) | 3.9 fps | ETA: 10.2min
Frame 450/2832 (15.9%) | 3.9 fps | ETA: 10.2min
Frame 480/2832 (16.9%) | 3.9 fps | ETA: 10.0min
Frame 510/2832 (18.0%) | 3.9 fps | ETA: 9.8min
Frame 540/2832 (19.1%) | 3.9 fps | ETA: 9.8min
Frame 570/2832 (20.1%) | 3.9 fps | ETA: 9.7min
Frame 600/2832 (21.2%) | 3.9 fps | ETA: 9.5m